In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, balanced_accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import logging
from datetime import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import mutual_info_classif

# Logging Setup
log_time = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"logs/{log_time}/model_evaluation.log"
os.makedirs(os.path.dirname(log_filename), exist_ok=True)
logging.basicConfig(filename=log_filename, level=logging.INFO)
print(f"Log file salvato in: {log_filename}\n")

def recode_labels_for_first_classifier(labels):
    return labels.apply(lambda x: "healthy" if x in [0, 1] else "infected")

def filter_data_for_second_classifier(X, y):
    mask = y.isin([2, 3, 4, 5, 6])
    return X[mask], y[mask]

def evaluate_model(model, X_test, y_test, model_name="Model"):
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")

    print(f"\n--- Evaluation: {model_name} ---")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(matrix)
    print(f"Balanced Accuracy: {balanced_acc}")
    print(f"F1 Macro: {f1_macro}")
    print("-" * 80)

    logging.info(f"\n--- Evaluation: {model_name} ---")
    logging.info(f"Classification Report:\n{report}")
    logging.info(f"Confusion Matrix:\n{matrix}")
    logging.info(f"Balanced Accuracy: {balanced_acc}")
    logging.info(f"F1 Macro: {f1_macro}")
    logging.info("-" * 80)

# Pipeline 1: XGBoost
def create_xgb_pipeline(num_class):
    pipeline = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42)),
        ('varthresh', VarianceThreshold(threshold=1e-5)),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif, k=100)),
        ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=num_class-1)),
        ('classifier', XGBClassifier(eval_metric='mlogloss', random_state=42))
    ])

    param_dist = {
        'selector__k': [700],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [3, 6, 10],
        'classifier__learning_rate': [0.01, 0.1, 0.2]
    }

    return pipeline, param_dist

# Pipeline 2: Random Forest
def create_rf_pipeline(num_class):
    pipeline = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42)),
        ('varthresh', VarianceThreshold(threshold=1e-5)),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif)),
        ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=num_class-1)),
        ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
    ])

    param_dist = {
        'selector__k': [700],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [None, 5, 10, 20],
        'classifier__min_samples_split': [2, 5, 10]
    }

    return pipeline, param_dist

# Pipeline 3: SVM
def create_svm_pipeline(num_class):
    pipeline = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42)),
        ('varthresh', VarianceThreshold(threshold=1e-5)),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif, k=100)),
        ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=num_class-1)),
        ('classifier', SVC(random_state=42, class_weight='balanced', kernel='linear'))
    ])

    param_dist = {
        'selector__k': [700],
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__kernel': ['linear', 'rbf', 'poly'],
        'classifier__gamma': ['scale', 'auto']  # Only relevant for rbf and poly kernels
    }
    return pipeline, param_dist

# Pipeline 4: Logistic Regression
def create_lr_pipeline(num_class):
    pipeline = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42)),
        ('varthresh', VarianceThreshold(threshold=1e-5)),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif, k=100)),
        ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=num_class-1)),
        ('classifier', LogisticRegression(random_state=42, class_weight='balanced'))
    ])

    param_dist = {
        'selector__k': [700],
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear', 'saga']
    }

    return pipeline, param_dist

# Pipeline 5: KNN
def create_knn_pipeline(num_class):
    pipeline = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42)),
        ('varthresh', VarianceThreshold(threshold=1e-5)),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(score_func=f_classif, k=100)),
        ('lda', LinearDiscriminantAnalysis(solver='svd', n_components=num_class-1)),
        ('classifier', KNeighborsClassifier())
    ])

    param_dist = {
        'selector__k': [700],
        'classifier__n_neighbors': [3, 5, 7, 9, 11],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__p': [1, 2]  # 1: manhattan, 2: euclidean
    }

    return pipeline, param_dist

def run_grid_search(pipeline, param_grid, X_train, y_train, model_name="Model"):
    print(f"Running GridSearchCV for {model_name} ...")
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring='f1_macro',
        n_jobs=2,
        cv=3,
        verbose=2,
        refit=True
    )
    grid_search.fit(X_train, y_train)
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best F1 Macro Score for {model_name}: {grid_search.best_score_:.4f}")
    logging.info(f"Best parameters for {model_name}: {grid_search.best_params_}")
    logging.info(f"Best F1 Macro Score for {model_name}: {grid_search.best_score_:.4f}")
    return grid_search.best_estimator_

def main():
    df = pd.read_csv("../roi_features_train.csv")
    X = df.drop(columns=["image_id", "score", "x1", "y1", "x2", "y2", "label"])
    y_original = df["label"]

    y_stage1 = recode_labels_for_first_classifier(y_original)
    le_stage1 = LabelEncoder()
    y_stage1_encoded = le_stage1.fit_transform(y_stage1)

    X_train_stage1, X_test_stage1, y_train_stage1, y_test_stage1 = train_test_split(
        X, y_stage1_encoded, test_size=0.2, stratify=y_stage1_encoded, random_state=42
    )

    pipelines = [
        #("XGBoost", create_xgb_pipeline),
        ("Random Forest", create_rf_pipeline),
        ("SVM", create_svm_pipeline),
        ("Logistic Regression", create_lr_pipeline),
        ("KNN", create_knn_pipeline)
    ]

     # Use fixed classifier for Stage 1
    name1 = "SVM"  # or "SVM", "Random Forest", etc.
    pipeline_func1 = dict(pipelines)[name1]
    print(f"\n{'='*40}\nGrid Search + Training Stage 1: Healthy vs Infected ({name1})\n{'='*40}")
    pipeline_stage1, param_dist_stage1 = pipeline_func1(num_class=2)

    # Run GridSearchCV
    best_pipeline_stage1 = run_grid_search(pipeline_stage1, param_dist_stage1, X_train_stage1, y_train_stage1, model_name=f"Stage 1 - {name1}")

    evaluate_model(best_pipeline_stage1, X_test_stage1, y_test_stage1, model_name=f"Stage 1 - {name1}")

    y_pred_stage1 = best_pipeline_stage1.predict(X_test_stage1)
    infected_indices = np.where(y_pred_stage1 == 1)[0]

    X_test_stage2 = X_test_stage1.iloc[infected_indices]
    allowed_labels = [2, 3, 4, 5, 6]
    mask_test_stage2 = y_original.iloc[X_test_stage2.index].isin(allowed_labels)
    X_test_stage2 = X_test_stage2[mask_test_stage2]
    y_test_stage2 = y_original.iloc[X_test_stage2.index]

    X_train_stage2, y_train_stage2 = filter_data_for_second_classifier(
        X_train_stage1, y_original.iloc[X_train_stage1.index]
    )

    le_stage2 = LabelEncoder()
    y_train_stage2_encoded = le_stage2.fit_transform(y_train_stage2)
    try:
        y_test_stage2_encoded = le_stage2.transform(y_test_stage2)
    except ValueError:
        unseen = set(y_test_stage2) - set(le_stage2.classes_)
        print(f"Skipping Stage 2 ({name1}) due to unseen labels in test: {unseen}")
        

    num_classes_stage2 = len(le_stage2.classes_)

    for name2, pipeline_func2 in pipelines:
        print(f"\n{'='*40}\nGrid Search + Training Stage 2: Infected Subtype Classification ({name1} ➡ {name2})\n{'='*40}")
        pipeline_stage2, param_dist_stage2 = pipeline_func2(num_class=num_classes_stage2)

        # Run GridSearchCV
        best_pipeline_stage2 = run_grid_search(pipeline_stage2, param_dist_stage2, X_train_stage2, y_train_stage2_encoded, model_name=f"Stage 2 - {name1} ➡ {name2}")

        evaluate_model(best_pipeline_stage2, X_test_stage2, y_test_stage2_encoded, model_name=f"Stage 2 - {name1} ➡ {name2}")

    print("✅ Pipeline with Grid Search completed successfully.")

if __name__ == "__main__":
    main()


Log file salvato in: logs/20250703_120727/model_evaluation.log


Grid Search + Training Stage 1: Healthy vs Infected (SVM)
Running GridSearchCV for Stage 1 - SVM ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=linear, selector__k=700; total time=  56.0s
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=linear, selector__k=700; total time=  56.8s
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=linear, selector__k=700; total time=  51.8s
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=rbf, selector__k=700; total time=  52.3s
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=rbf, selector__k=700; total time=  50.0s
[CV] END classifier__C=0.001, classifier__gamma=scale, classifier__kernel=rbf, selector__k=700; total time=  53.7s


/home/achille/Desktop/ML/.venv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
